In [34]:
import openpyxl   
import pandas as pd
import numpy as np
from pandas import read_excel
import matplotlib.pyplot as plt
import scipy.optimize as scp

In [87]:
raw_data=pd.read_excel("air1.xlsx", engine = 'openpyxl') 
data = pd.DataFrame(raw_data)
data = data.drop([0,1]).reset_index(drop=True)
data

,Unnamed: 0,Sample,T,ch1,ch2,mean,inverted
0,2,-329,-0.000007,0.000801,0.000297,0.000221,-0.000221
1,3,-328,-0.000007,0.000801,0.000000,0.000220,-0.000220
2,4,-327,-0.000007,0.000801,0.000099,0.000245,-0.000245
3,5,-326,-0.000007,0.000801,0.000198,0.000194,-0.000194
4,6,-325,-0.000006,0.000801,0.000397,0.000245,-0.000245
...,...,...,...,...,...,...,...
131065,131067,130736,0.002615,0.000801,0.000099,0.000225,-0.000225
131066,131068,130737,0.002615,0.000601,0.000099,0.000222,-0.000222
131067,131069,130738,0.002615,0.001002,0.000595,0.000216,-0.000216
131068,131070,130739,0.002615,0.001002,0.000198,0.000217,-0.000217


In [88]:
time_column = data.columns[2]
ch1_column = data.columns[3]
data_column = data.columns[-1]
print(time_column,ch1_column,data_column)
#print(np.average(data[ch1_column])*10)

%matplotlib widget
plt.plot(data[time_column],data[ch1_column])
plt.plot(data[time_column],data[data_column], color='r')
plt.show()


T ch1 inverted


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
tester = []
for i in range(len(data[ch1_column])):
    tester.append(data[ch1_column].to_numpy()[i]-data[ch1_column].to_numpy()[i-1])

thershold =  (np.max(tester)-np.average(tester))/3.5
print(thershold)

%matplotlib widget
plt.plot(data[time_column], tester)
plt.axhline(thershold, color='r')
#plt.plot(data[time_column],data[data_column], color='r')

0.004778596571428572


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
def f(x,a,b):
    return a*np.exp(-x/b)
    

In [90]:
points_candidates = np.argwhere(tester>thershold)
points_candidates = np.append(points_candidates,[[(len(data[ch1_column])-1)]], axis=0)
#points_candidates.append()

i = 0
points = []
while i<(len(points_candidates)-1):
    distance = points_candidates[i+1][0]-points_candidates[i][0]
    if distance < 200:
        if data[ch1_column][points_candidates[i][0]]>data[ch1_column][points_candidates[i+1][0]]:
            points.append(points_candidates[i][0])
        else:
            points.append(points_candidates[i+1][0])
        i = i + 1

    else:
        points.append(points_candidates[i][0])

    i = i + 1

points.append(len(data[ch1_column])-1)
#print(points)
#print(points_candidates)


%matplotlib widget
plt.plot(data[time_column],data[data_column])


for i in range(len(points)):
    plt.axvline(x= data[time_column][points[i]], color = 'r')

"""

for i in range(len(points_candidates)):
    plt.axvline(x= data[time_column][points_candidates[i][0]], color = 'g')

"""

plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:

thaus = []

for i in range(len(points)-1):

    indexes = np.arange(points[i],points[i+1])
    val_between_imp = data[data_column].to_numpy()[indexes]
    peak = np.argmax(val_between_imp)

    times_fit = np.array(data[time_column ].to_numpy()[indexes][peak:] - data[time_column ].to_numpy()[indexes][peak:][0],dtype=np.float32)
    val_fit = np.array((data[data_column].to_numpy()[indexes][peak:]-np.average(data[data_column].to_numpy()[indexes][peak:][-80:])),dtype=np.float32)
    a_init = val_fit[0]
    b_init = -1/(np.log(val_fit[80]/a_init)/times_fit[80])

    params, pcov = scp.curve_fit(f, times_fit, val_fit, [a_init,b_init])
    err = np.sqrt(np.diag(pcov))

    thaus.append(params[1])
    
    """
    %matplotlib widget
    plt.plot(times_fit,val_fit)
    plt.plot(times_fit, f(times_fit, params[0], params[1]), color = 'r')
    """
print(np.average(thaus),np.std(thaus))






2.3222715949271765e-06 6.86183394638994e-08


In [ ]:
nitrogen:
2.362955057402244e-06 5.691787963020979e-08
air1:
2.3222715949271765e-06 6.86183394638994e-08
air2:
2.2920622812734162e-06 8.528731250173426e-08
crone1:



